# Missing Data and Inconsistent coding

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

sns.set_theme(f palette= "colorblind")
na_toy_df = pd.DataFrame(data = [[1,3,4,5],[2 ,6, np.nan]])

# make plots look nicer and increase font size
sns.set_theme(font_scale=2,palette='colorblind')
arabica_data_url = 'https://raw.githubusercontent.com/jldbc/coffee-quality-database/master/data/arabica_data_cleaned.csv'

coffee_df = pd.read_csv(arabica_data_url)


rhodyprog4ds_gh_events_url = 'https://api.github.com/orgs/rhodyprog4ds/events'
course_gh_df = pd.read_json(rhodyprog4ds_gh_events_url)

SyntaxError: invalid syntax (1202045574.py, line 5)

So far, we've dealt with structural issues in data. but there's a lot more to
cleaning.  

Today,  we'll deal with how to fix the values wihtin  the data.  To see the
types of things:

[Stanford Policy Lab Open Policing Project data readme](https://github.com/stanford-policylab/opp/blob/master/data_readme.md)
[Propublica Machine Bias](https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm) the "How we acquired data" section

## Missing Values


Dealing with missing data is a whole research area. There isn't one solution.

[in 2020 there was a workshop on it](https://artemiss-workshop.github.io/)

There are also many classic approaches both when training and when [applying models](https://www.jmlr.org/papers/volume8/saar-tsechansky07a/saar-tsechansky07a.pdf).

[example application in breast cancer detection](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.701.4234&rep=rep1&type=pdf)

In pandas, even representing  [missing values](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html) is under [experimentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html#missing-data-na). Currently, it uses `numpy.NaN`, but the experiment is with `pd.NA`.

Missing values even causes the [datatypes to change](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html#missing-data-casting-rules-and-indexing)

Pandas gives a few basic tools:
 - drop with (`dropna`)
 - fill with `fillna`

In [2]:
coffee_df.head()

NameError: name 'coffee_df' is not defined

The 'Lot.Number' has a lot of NaN values, how can we explore it?

We can look at the type:

In [3]:
coffee_df['Lot.Number'].dtype

NameError: name 'coffee_df' is not defined

And we can look at the value counts.

In [4]:
coffee_df['Lot.Number'].value_counts()

NameError: name 'coffee_df' is not defined

Filling can be good if you know how to fill reasonably, but don't have data to
spare by dropping.  For example
- you can approximate with another column
- you can approximate with that column from other rows



We see that a lot are '1', maybe we know that when the data was collected, if the Farm only has one lot, some people recorded '1' and others left it as missing. So we could fill in with 1:

In [5]:
coffee_df['Lot.Number'].fillna(1).head()

NameError: name 'coffee_df' is not defined

In [6]:
coffee_df['Lot.Number'].head()

NameError: name 'coffee_df' is not defined

```{tip}
Note that even after we called `fillna` we display it again and the original data is unchanged.
```

To save the filled in column we have a few choices:
- use the `inplace` parameter. This doesn't offer performance advantages, but does It still copies the object, but then reassigns the pointer. Its under discussion to [deprecate](https://github.com/pandas-dev/pandas/issues/16529)
- write to a new DataFrame
- add a column

We'll use adding a column:

In [7]:
coffee_df['lot_number_clean'] = coffee_df['Lot.Number'].fillna(1)

NameError: name 'coffee_df' is not defined

```{admonition} Question in Class
When I use value counts it treats the filled ones as different. Why?

```

In [8]:
coffee_df['Lot.Number'].value_counts()

NameError: name 'coffee_df' is not defined

In [9]:
coffee_df['lot_number_clean'].value_counts()

NameError: name 'coffee_df' is not defined

If we swithc to `1` as a string, then we'd see all of the one values as the same thing.

In [10]:
coffee_df['lot_number_clean'] = coffee_df['Lot.Number'].fillna('1')

coffee_df['lot_number_clean'].value_counts()

NameError: name 'coffee_df' is not defined

This was our goal, so in this case, it's the right thing to do to overwrite the value.



Dropping is a good choice when you otherwise have a lot of data and the data is
missing at random.

Dropping can be risky if it's not missing at random. For example, if we saw in
the coffee data that one of the scores was missing for all of the rows from one
country, or even just missing more often in one country, that could bias our
results.

To illustrate how `dropna` works, we'll use the `shape` method:

In [11]:
coffee_df.shape

NameError: name 'coffee_df' is not defined

By default, it drops any row with one or more `NaN` values.

In [12]:
coffee_df.dropna().shape

NameError: name 'coffee_df' is not defined

We could instead tell it to only drop rows with `NaN` in a subset of the columns.

````{margin}
```{note}
subset operates along columns by default, because axis is set to 0, by default.
```
````

In [13]:
coffee_df.dropna(subset=['altitude_low_meters']).shape

NameError: name 'coffee_df' is not defined

**whatever you do, document it**

```{adomonition} Try it Yourself
use the `na_toy_df` DataFrame that's defined in the first cell, to experiment with subset and axis parameters to understand them better.
```

In the [Open Policing Project Data Summary](https://openpolicing.stanford.edu/data/) we saw that they made a summary information that showed which variables had at least 70% not missing values.  We can similarly choose to keep only variables that have more than a specific threshold of data, using the `thresh` parameter and `axis=1` to drop along columns.

In [14]:
n_rows, n_cols = coffee_df.shape

coffee_df.dropna(thresh=.7*n_rows,axis=1).shape

NameError: name 'coffee_df' is not defined

This dataset is actually in pretty good shape, but if we use a more stringent threshold it drops more columns.

In [15]:
coffee_df.dropna(thresh=.85*n_rows,axis=1).shape

NameError: name 'coffee_df' is not defined

```{important}
Everththing after this is new material that we did not have time for in class,
but is important and helpful in your assignment (and for your portflio).
```

## Inconsistent values

This was one of the things that many of you anticipated or had observed.  A useful way to investigate for this, is to use `value_counts` and sort them alphabetically by the values from the original data, so that similar ones will be consecutive in the list. Once we have the `value_counts()` Series, the values from the `coffee_df` become the index, so we use `sort_index`.

Let's look at the `In.Country.Partner` column

In [16]:
coffee_df['In.Country.Partner'].value_counts().sort_index()

NameError: name 'coffee_df' is not defined

We can see there's only one `Blossom Valley International\n` but 58 `Blossom Valley International`, the former is likely a typo, especially since `\n` is a special character for a newline. Similarly, with 'Specialty Coffee Ass' and 'Specialty Coffee Association'.

This is another job for dictionaries, we make one with the value to replace as the key and the value to insert as the value.

In [17]:
partner_corrections = {'Blossom Valley International\n':'Blossom Valley International',
  'Specialty Coffee Ass':'Specialty Coffee Association'}
coffee_df['in_country_partner_clean'] = coffee_df['In.Country.Partner'].replace(
  to_replace=partner_corrections)
coffee_df['in_country_partner_clean'].value_counts().sort_index()

NameError: name 'coffee_df' is not defined

and now we see the corrected values.  We can also pass lambdas or put lambas in the dictionary if there are systemic patterns.

## Fixing data at load time

Explore some of the different parameters in `read_csv`


How can we read in data that looks like this:
![mulitindex img of excel file](https://github.com/rhodyprog4ds/BrownFall20/raw/main/img/multiindex.png)

```
pd.read_csv('fancy_formatting.xlsx', header = list(range(4)))
```

Many problems can  be repaired with parameters in [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html#pandas.read_csv).

## A Cleaning Data Recipe

__not everything possible, but good enough for this course__


1. Can you use parameters to read the data in better?
1. Fix the index and column headers (making these easier to use makes the rest easier)
1. Is the data structured well?
1. Are there missing values?
1. Do the datatypes match what you expect by looking at the head or a sample?
1. Are categorical variables represented in usable way?
1. Does your analysis require filtering or augmenting the data?

Things to keep in mind:
- always save new copies of data when you mutate it
- add new columns rather than overwriting columns
- long variable names are better than ambiguous naming


## Your observations from Monday:

I promised we'd come back to your observations on what problems could occur in
data. Here they are, organized by rough categories of when/how to fix them.


We can fix while reading in data:
- decimal was indicated with ',' insead of '.' so pandas saw value as a string rather than a float
- missing header
- reading the index as a column
- large datasets might be too slow or not fit in memory
- missing data represeted with a value or special character

We can fix by reshaping data:
- Data can get read into tables in bizarre ways depending on how the data was entered originally.
- every value in one column, instead of separated

We can repair by changing values or filtering:
- information represented inconsistently eg "Value" and " Value " or twenty-two instead of 22
- blank rows or blank columns or data that is N/A
- date/time information can be represented lots of different ways
- representing categorical with numbers that are ambiguous
- spaces or other symbols in column names
- some numbers as strings, others as ints within a column
- symbols being mis interpreted


Real problems, but beyond our scope:
- corrupt data files


## More Practice

Instead of more practice with these manipulations, below are more
examples of cleaning data to see how these types of manipulations get used.  
Your goal here is not to memorize every possible thing, but to build a general
idea of what good data looks like and good habits for cleaning data and keeping
it reproducible.  
- [Cleaning the Adult Dataset](https://ryanwingate.com/projects/machine-learning-data-prep/adult/adult-cleaning/)
- [All Shades](https://github.com/the-pudding/data/tree/master/foundation-names#allshadescsv--allshadesr)

Also here are some tips on general data management and organization.

This article is a comprehensive [discussion of data cleaning](https://towardsdatascience.com/the-ultimate-guide-to-data-cleaning-3969843991d4).